# Installing the Essential Libraries

In [74]:
# !pip install PyPDF2
# !pip install langchain
# !pip install groq
# !pip install -U langchain-community
# !pip install transformers
# !pip install faiss-cpu

# Importing Libraries

In [12]:
import os
import PyPDF2
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from google.colab import files
from transformers import AutoTokenizer, AutoModel
import torch

In [5]:
os.environ["GROQ_API_KEY"] = groq_api_key

In [30]:
from google.colab import files
uploaded = files.upload()

Saving IITM BS Degree Programme - Student Handbook-1-10.pdf to IITM BS Degree Programme - Student Handbook-1-10.pdf


# Converting PDF to Text

In [31]:
def pdf_to_text(pdf_path):
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

In [32]:
pdf_path = list(uploaded.keys())[0]
pdf_text = pdf_to_text(pdf_path)

# Converting the Entire Text to Chunks

In [33]:
text_splitter = CharacterTextSplitter(chunk_size=4000, chunk_overlap=1000)
chunks = text_splitter.split_text(pdf_text)

In [18]:
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-18-3756ac881042>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Creating the Vector Store from the text in terms of chunks taking the help of HF embeddings.

In [34]:
vector_store = FAISS.from_texts(chunks, hf_embeddings)

In [20]:
system_prompt = """
You are an AI assistant trained to answer questions based on the content of the PDF document provided, which is a student handbook.
The document is highly detailed and contains knowledge about the entire IIT Madras BS Degree.
When asked a question, you should refer to the document and provide the most accurate and relevant answer based on the information in the document.
Answer only based on the content from the document and do not make up any information.
"""


# Preparing the Model with RAG

In [53]:
from groq import Groq

client = Groq(api_key=groq_api_key)

def query_model_with_rag(query):
    relevant_chunks = vector_store.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in relevant_chunks])
    prompt = system_prompt + "\n\nContext:\n" + context + "\n\nQuestion: " + query + "\nAnswer:"
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama-3.3-70b-versatile",
        stream=False
    )
    return chat_completion.choices[0].message.content.strip()

In [49]:
from pprint import pprint

# Evaluation

In [54]:
query = "What is the total amount of credits required to pass the BS Degree?"
answer = query_model_with_rag(query)
print(answer)

The total amount of credits required to graduate with the BS degree is 142.


In [47]:
query = "What is the fee of level 3 courses?"
answer = query_model_with_rag(query)
print(answer)

The fee for the level 3 courses is Rs 2500/credit.


In [50]:
query = "How can I get minor in Economics and Finance"
answer = query_model_with_rag(query)
pprint(answer)

('To get a minor in Economics and Finance, you need to complete 2 mandatory '
 'courses: Corporate Finance and Managerial Economics, and one elective course '
 'from the list provided. These 3 courses can be credited within the 142 '
 'credits required for the BS degree. The Minor can be earned only with the BS '
 'degree and not the BSc degree. A separate document will be issued by IITM '
 'stating that the Minor has been completed, but there will be no change in '
 'the transcript or the degree certificate for those who do the minor.')
